In [1]:
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=UserWarning)

In [2]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

sia = SentimentIntensityAnalyzer()

# This Hugging Face Transformers library to load a pre-trained sentiment analysis model for Twitter data.
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL) # Convert the raw text into a format so that the model can process
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abirp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
with open('TwitterSentimentModel', 'rb') as f:
  myModel = pickle.load(f)

In [5]:
knn = myModel['knn']
lr = myModel['lr'] # Logistic Regression
rf = myModel['rf']
mlr = myModel['mlr'] # Multiple Linear Regrssion 

In [6]:
def getAllPolarityScore(text):
  vadarScore = sia.polarity_scores(text)
  encoded_text = tokenizer(text,return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  scores_dict = {
      "roberta_neg" : scores[0],
      "roberta_neu" : scores[1],
      "roberta_pos" : scores[2]
  }
  vadarList =  [i for i in vadarScore.values()]
  robertaList =[i for i in scores_dict.values()]
  knn_predict = knn.predict([(vadarList + robertaList)])
  linearReggPredict = mlr.predict([(vadarList + robertaList)])
  logisticReggPredict = lr.predict([(vadarList + robertaList)])
  randomForestPredict = rf.predict([(vadarList + robertaList)])

  return {'TAG' : { "KNN":knn_predict,"RandomForest":randomForestPredict,"Logistic Regression":logisticReggPredict }, 'SCORE' :  linearReggPredict }


In [7]:
getAllPolarityScore("I am happy")

{'TAG': {'KNN': array([1], dtype=int64),
  'RandomForest': array([1], dtype=int64),
  'Logistic Regression': array([1], dtype=int64)},
 'SCORE': array([0.98189332])}

In [8]:
getAllPolarityScore("I am Sad")

{'TAG': {'KNN': array([0], dtype=int64),
  'RandomForest': array([0], dtype=int64),
  'Logistic Regression': array([0], dtype=int64)},
 'SCORE': array([0.99357934])}

In [9]:
getAllPolarityScore("G20 finance meet ends, no consensus on Ukraine")

{'TAG': {'KNN': array([0], dtype=int64),
  'RandomForest': array([0], dtype=int64),
  'Logistic Regression': array([0], dtype=int64)},
 'SCORE': array([0.94332673])}

In [10]:
getAllPolarityScore("‘India ready to contribute to peace process’: PM Modi on Ukraine crisis")

{'TAG': {'KNN': array([1], dtype=int64),
  'RandomForest': array([1], dtype=int64),
  'Logistic Regression': array([1], dtype=int64)},
 'SCORE': array([0.91550444])}